# Artificial Neural Networks and Deep Learning

---

## SiumGPT Homework 2 Final Notebook (Models Ensemble)

## ⚙️ Import Libraries

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'
os.environ["SM_FRAMEWORK"] = "tf.keras"
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np
import pandas as pd
from datetime import datetime

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import albumentations as A
import os
import math
from PIL import Image
from keras import backend as K
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from keras.utils import to_categorical
from keras.saving import load_model
from keras.metrics import MeanIoU
from keras import saving as ks

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

!pip install -U segmentation-models
import segmentation_models as sm

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

2.16.1
TensorFlow version: 2.16.1
Keras version: 3.3.3
GPU devices: 1


## ⏳ Load the Data

In [ ]:
data = np.load("/kaggle/input/mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

# not used, just as a reference
labels = {
    0: "Background",
    1: "Soil",
    2: "Bedrock",
    3: "Sand",
    4: "Big Rock"
}

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)


In [ ]:
# Removing aliens
alien_label = y_train[1834, :]
#plt.imshow(alien_label, cmap='viridis')

filter = [True] * X_train.shape[0]
removed = 0
for i, lab in enumerate(y_train):
    if np.array_equal(alien_label, lab):
        filter[i] = False
        removed += 1

X_train = X_train[filter]
y_train = y_train[filter]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f'Removed {removed} images')

Training X shape: (2505, 64, 128)
Training y shape: (2505, 64, 128)
Removed 110 images


In [ ]:
# Add color channel
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Input shape: (64, 128, 1)
Number of classes: 5


In [ ]:
def apply_augmentations(images, masks, aug_list):
    """
    Applies a list of augmentations to images and masks, returning the augmented data.

    Args:
        images (np.ndarray): Array of input images.
        masks (np.ndarray): Array of corresponding ground truth masks.
        aug_list (list): List of augmentation functions to apply.

    Returns:
        dict: Dictionary containing augmented images under key "images" and augmented masks under key "labels".
    """
    augmented_images = []
    augmented_masks = []

    for aug in aug_list:
        for img, mask in zip(images, masks):
            augmented = aug(image=img, mask=mask)
            augmented_images.append(augmented['image'])
            augmented_masks.append(augmented['mask'])

    augmented_images = np.array(augmented_images)
    augmented_masks = np.array(augmented_masks)

    return {"images":augmented_images, "labels":augmented_masks}

# Augmentations
h_flip = A.HorizontalFlip(p=1.0)
v_flip = A.VerticalFlip(p=1.0)
rotation = A.Affine(rotate=180,p=1)
augmentations = [h_flip, v_flip, rotation]
aug_dataset = apply_augmentations(X_train, y_train, augmentations)
X_train, y_train = aug_dataset['images'], aug_dataset['labels']

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")

Training X shape: (7515, 64, 128, 1)
Training y shape: (7515, 64, 128)


In [ ]:
# Splitting in train-validation sets
train_img, val_img, train_lbl, val_lbl = train_test_split(
    X_train, y_train, test_size=0.1, random_state=seed
)
print("Data splitted!")

train_lbl_cat = tf.one_hot(train_lbl, depth=num_classes)
val_lbl_cat = tf.one_hot(val_lbl, depth=num_classes)

print(f"\nNumber of images:")
print(f"Train: {len(train_img)}")
print(f"Validation: {len(val_img)}")
print(f"\nLabels shape:")
print(f"Train: {train_lbl_cat.shape}")
print(f"Validation: {val_lbl_cat.shape}")

Data splitted!

Number of images:
Train: 6763
Validation: 752

Labels shape:
Train: (6763, 64, 128, 5)
Validation: (752, 64, 128, 5)


## 🛠️ Model Ensemble

In [ ]:
@ks.register_keras_serializable()
class EnsembleModel(tfk.Model):
    """
    Custom ensemble model that combines predictions from multiple models using weighted averaging.

    Args:
        models (list): List of pre-trained models to include in the ensemble.
        weights (list or np.array): Weights for combining the predictions of each model.
    """
    def __init__(self, models, weights):
        super(EnsembleModel, self).__init__()
        self.models = models  # List of models
        self.ensemble_weights = tf.constant(weights, dtype=tf.float32)  # Ensemble weights

    def call(self, inputs):
        # Collect predictions from each model
        predictions = [model(inputs) for model in self.models]
        predictions = tf.stack(predictions, axis=0)  # Stack along a new dimension

        # Compute weighted predictions
        weighted_predictions = tf.tensordot(predictions, self.ensemble_weights, axes=((0), (0)))
        return tf.argmax(weighted_predictions, axis=-1) # The argmax is kept inside the class code, because we don't need this model for training only inference

    def get_config(self):
        # Save model configurations and weights for reloading
        return {
            "models": [model.to_json() for model in self.models],  # Save each model structure as JSON
            "weights": self.ensemble_weights.numpy().tolist(),  # Convert tensor to Python list for serialization
        }

    @classmethod
    def from_config(cls, config):
        # Recreate models from JSON and reload weights
        models = [tfk.models.model_from_json(model_json) for model_json in config["models"]]
        weights = config["weights"]
        return cls(models=models, weights=weights)

In [ ]:
#Set compile=False as we are not loading it for training, only for prediction.
model1 = load_model('/kaggle/input/model1.keras', compile=False)
model2 = load_model('/kaggle/input/model2.keras', compile=False)

models = [model1, model2]

In [ ]:
def calculate_classwise_miou(y_true, y_pred, num_classes):
    """
    Calculate the mean Intersection over Union (mIoU) for each class.

    Args:
        y_true (np.array): Ground truth labels (e.g., shape: (batch, height, width)).
        y_pred (np.array): Predicted labels (e.g., shape: (batch, height, width)).
        num_classes (int): Total number of classes.

    Returns:
        dict: Dictionary of IoU values for each class.
    """
    # Flatten the arrays
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # Initialize the MeanIoU object
    miou_metric = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0,)
    miou_metric.update_state(y_true_flat, y_pred_flat)

    # Extract IoU per class
    total_conf_matrix = miou_metric.total_cm.numpy()  # Get the confusion matrix
    ious = []
    for i in range(num_classes):
        TP = total_conf_matrix[i, i]  # True positives for class i
        FP = total_conf_matrix[:, i].sum() - TP  # False positives for class i
        FN = total_conf_matrix[i, :].sum() - TP  # False negatives for class i
        denominator = TP + FP + FN
        if denominator == 0:
            iou = np.nan  # Handle classes not present in predictions or labels
        else:
            iou = TP / denominator
        ious.append(iou)

    # Create a dictionary for class-wise IoU
    miou_per_class = {f"Class {i}": iou for i, iou in enumerate(ious)}
    return miou_per_class

val_lbl_pred1 = model1.predict(val_img)
val_lbl_pred2 = model2.predict(val_img)

y_pred_argmax1 = np.argmax(val_lbl_pred1, axis=-1)  # Convert one-hot predictions to class indices
y_pred_argmax2 = np.argmax(val_lbl_pred2, axis=-1)  # Convert one-hot predictions to class indices
val_lbl_argmax = np.argmax(val_lbl_cat, axis=-1)  # Convert one-hot labels to class indices

miou_per_class1 = calculate_classwise_miou(val_lbl_argmax, y_pred_argmax1, num_classes)
miou_per_class2 = calculate_classwise_miou(val_lbl_argmax, y_pred_argmax2, num_classes)

# Print the IoU scores for each class
print("Class-wise IoU scores for model1:")
for cls, score in miou_per_class1.items():
    print(f"{cls}: {score:.4f}")

# Print the IoU scores for each class
print("Class-wise IoU scores for model2:")
for cls, score in miou_per_class2.items():
    print(f"{cls}: {score:.4f}")

I0000 00:00:1734210499.807522     138 service.cc:145] XLA service 0x7c08f0003040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734210499.807607     138 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 8/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

I0000 00:00:1734210505.010945     138 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 244ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step
Class-wise IoU scores for model1:
Class 0: nan
Class 1: 0.8558
Class 2: 0.8084
Class 3: 0.8655
Class 4: 0.3231
Class-wise IoU scores for model2:
Class 0: nan
Class 1: 0.8864
Class 2: 0.8268
Class 3: 0.8921
Class 4: 0.0000


In [ ]:
# Best ensemble model search
best = dict({'wts': [0.0, 0.0], 'IOU': 0.0, 'model': None}, index=[0])
for w1 in range(1,10):
    wts = [w1/10.,0.1]
    ensemble_model = EnsembleModel(models, wts)
    IOU_wted = MeanIoU(num_classes=num_classes, ignore_class=0)
    wted_ensemble_pred = ensemble_model.predict(val_img)
    IOU_wted.update_state(val_lbl, wted_ensemble_pred)
    print("Now predicting for weights :", w1/10., 0.1, " : IOU = ", IOU_wted.result().numpy())
    if IOU_wted.result().numpy() > best.get('IOU'):
        best = dict({'wts' : [wts[0], wts[1]],'IOU': IOU_wted.result().numpy(), 'model': ensemble_model}, index=[0])
for w2 in range(1,10):
    wts = [0.1,w2/10.]
    ensemble_model = EnsembleModel(models, wts)
    IOU_wted = MeanIoU(num_classes=num_classes, ignore_class=0)
    wted_ensemble_pred = ensemble_model.predict(val_img)
    IOU_wted.update_state(val_lbl, wted_ensemble_pred)
    print("Now predicting for weights :", 0.1, w2/10., " : IOU = ", IOU_wted.result().numpy())
    if IOU_wted.result().numpy() > best.get('IOU'):
        best = dict({'wts' : [wts[0], wts[1]],'IOU': IOU_wted.result().numpy(), 'model': ensemble_model}, index=[0])

print("Best weights found: ", best.get('wts'), " : IOU = ", best.get('IOU'))

24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 301ms/step
Now predicting for weights : 0.1 0.1  : IOU =  0.73554194
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 276ms/step
Now predicting for weights : 0.2 0.1  : IOU =  0.7204041
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 277ms/step
Now predicting for weights : 0.3 0.1  : IOU =  0.717947
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 284ms/step
Now predicting for weights : 0.4 0.1  : IOU =  0.7168317
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 273ms/step
Now predicting for weights : 0.5 0.1  : IOU =  0.7161571
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 276ms/step
Now predicting for weights : 0.6 0.1  : IOU =  0.71564746
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 283ms/step
Now predicting for weights : 0.7 0.1  : IOU =  0.7153106
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 273ms/step
Now predicting for weights : 0.8 0.1  : IOU =  0.7150263
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 331ms/step
Now predicting for weights : 0.9 0.1  : IOU =  0.7148258
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 279ms/step
Now predicting for weights : 0.1 0.1  : IOU =  0.73554194
24/24 ━━

In [ ]:
pred1 = model1.predict(val_img)
pred2 = model2.predict(val_img)

y_pred1_argmax=np.argmax(pred1, axis=3)
y_pred2_argmax=np.argmax(pred2, axis=3)

ensemble_input = tfk.Input(shape=input_shape)
ensemble_model = best.get('model')
del best
ensemble_predictions = ensemble_model.predict(val_img)

IOU1 = MeanIoU(num_classes=num_classes, ignore_class=0)
IOU2 = MeanIoU(num_classes=num_classes, ignore_class=0)
IOU_weighted = MeanIoU(num_classes=num_classes, ignore_class=0)

IOU1.update_state(val_lbl, y_pred1_argmax)
IOU2.update_state(val_lbl, y_pred2_argmax)
IOU_weighted.update_state(val_lbl, ensemble_predictions)

print('IOU Score for model1 = ', IOU1.result().numpy())
print('IOU Score for model2 = ', IOU2.result().numpy())
print('IOU Score for weighted average ensemble = ', IOU_weighted.result().numpy())

miou_per_class = calculate_classwise_miou(val_lbl_argmax, ensemble_predictions, num_classes)

# Print the IoU scores for each class
print("Class-wise IoU scores:")
for cls, score in miou_per_class.items():
    print(f"{cls}: {score:.4f}")

timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
ensemble_model.save(model_filename)

print(f"Model saved to {model_filename}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
IOU Score for model1 =  0.71318656
IOU Score for model2 =  0.65132415
IOU Score for weighted average ensemble =  0.73554194
Class-wise IoU scores:
Class 0: nan
Class 1: 0.8829
Class 2: 0.8401
Class 3: 0.8928
Class 4: 0.3264
Model saved to model_241214_211228.keras


## 📊 Prepare Your Submission

In our Kaggle competition, submissions are made as `csv` files. To create a proper `csv` file, you need to flatten your predictions and include an `id` column as the first column of your dataframe. To maintain consistency between your results and our solution, please avoid shuffling the test set. The code below demonstrates how to prepare the `csv` file from your model predictions.



In [ ]:
# If model_filename is not defined, load the most recent model from Google Drive
if "model_filename" not in globals() or model_filename is None:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('model_') and f.endswith('.keras')]
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    if files:
        model_filename = files[0]
    else:
        raise FileNotFoundError("No model files found in the current directory.")

In [ ]:
preds = ensemble_model.predict(X_test)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 25s 61ms/step
Predictions shape: (10022, 64, 128)


In [ ]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [ ]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

%cd /kaggle/working
from IPython.display import FileLink
FileLink(submission_filename)

/kaggle/working


/kaggle/working/submission_241214_211228.csv

#  
<img src="https://airlab.deib.polimi.it/wp-content/uploads/2019/07/airlab-logo-new_cropped.png" width="350">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Instagram_logo_2022.svg/800px-Instagram_logo_2022.svg.png" width="15"> **Instagram:** https://www.instagram.com/airlab_polimi/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/LinkedIn_icon.svg/2048px-LinkedIn_icon.svg.png" width="15"> **LinkedIn:** https://www.linkedin.com/company/airlab-polimi/